In [1]:
import os
import sys

In [2]:
%pwd

'e:\\Projects\\MLproject-2_Kidney-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Projects\\MLproject-2_Kidney-Disease-Classification'

In [5]:
# Add src directory to sys.path
sys.path.append("E:\\Projects\\MLproject-2_Kidney-Disease-Classification\\src")

# Verify the path is added
print("Updated Python Path:", sys.path)

Updated Python Path: ['e:\\Projects\\MLproject-2_Kidney-Disease-Classification\\kidney\\python38.zip', 'e:\\Projects\\MLproject-2_Kidney-Disease-Classification\\kidney\\DLLs', 'e:\\Projects\\MLproject-2_Kidney-Disease-Classification\\kidney\\lib', 'e:\\Projects\\MLproject-2_Kidney-Disease-Classification\\kidney', '', 'e:\\Projects\\MLproject-2_Kidney-Disease-Classification\\kidney\\lib\\site-packages', 'e:\\Projects\\MLproject-2_Kidney-Disease-Classification\\kidney\\lib\\site-packages\\win32', 'e:\\Projects\\MLproject-2_Kidney-Disease-Classification\\kidney\\lib\\site-packages\\win32\\lib', 'e:\\Projects\\MLproject-2_Kidney-Disease-Classification\\kidney\\lib\\site-packages\\Pythonwin', 'e:\\Projects\\MLproject-2_Kidney-Disease-Classification\\kidney\\lib\\site-packages\\setuptools\\_vendor', 'E:\\Projects\\MLproject-2_Kidney-Disease-Classification\\src']


In [6]:
%pwd

'e:\\Projects\\MLproject-2_Kidney-Disease-Classification'

In [7]:
import setuptools
print(setuptools.find_packages(where="src"))

['cnnClassifier', 'cnnClassifier.components', 'cnnClassifier.config', 'cnnClassifier.constants', 'cnnClassifier.entity', 'cnnClassifier.pipeline', 'cnnClassifier.utils']


In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [11]:
!conda list

# packages in environment at e:\Projects\MLproject-2_Kidney-Disease-Classification\kidney:
#
# Name                    Version                   Build  Channel
absl-py                   2.1.0                    pypi_0    pypi
asttokens                 2.4.1              pyhd8ed1ab_0    conda-forge
astunparse                1.6.3                    pypi_0    pypi
backcall                  0.2.0              pyh9f0ad1d_0    conda-forge
beautifulsoup4            4.12.3                   pypi_0    pypi
ca-certificates           2024.8.30            h56e8100_0    conda-forge
cachetools                5.5.0                    pypi_0    pypi
certifi                   2024.8.30                pypi_0    pypi
charset-normalizer        3.4.0                    pypi_0    pypi
colorama                  0.4.6              pyhd8ed1ab_0    conda-forge
comm                      0.2.2              pyhd8ed1ab_0    conda-forge
debugpy                   1.6.7            py38hd77b12b_0  
decorator          

In [12]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [13]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)

    

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



In [14]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-11-26 11:58:41,105: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-26 11:58:41,205: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-26 11:58:41,299: INFO: common: created directory at: artifacts]
[2024-11-26 11:58:41,367: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-11-26 11:58:50,669: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64